In [1]:
reset -f

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
from ipyleaflet import Map, GeoData, basemaps, LayersControl, Marker, Popup, FullScreenControl
#import json
import ipywidgets as ipyw

In [3]:
gdf = gpd.read_file('data/cb_2019_06_tract_500k/cb_2019_06_tract_500k.shp')

In [4]:
# get screwed (replace with real metrics later)
screwed_dummy = gdf.sample(740)

In [5]:
def add_screwed(row):
    if row['GEOID'] in list(screwed_dummy['GEOID']):
        val = 1
    else:
        val = 0
    return(val)


def add_color(row):
    if row['screwed'] == 1:
        val = '#34eb4f'
    else:
        val = '#eb7a34'
    return(val)


In [6]:
gdf['screwed'] = gdf.apply(add_screwed, axis=1)
# gdf['color'] = gdf.apply(add_color, axis=1)

In [7]:
# split dataframes into nonscrewed and screwed
gdf_screwed = gdf[gdf['screwed'] == 1]
gdf_therest = gdf[gdf['screwed'] == 0]
# print((len(gdf)) == (len(gdf_therest) + (len(gdf_screwed))))

In [8]:
# prepping map
zoom = 6
center = [37.248, -119.432]

In [9]:
# prepping layers for map
ct_therest = GeoData(geo_dataframe = gdf_therest,
                  style={'color': 'black', 'fillColor': '#34eb4f', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                  hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                  name = 'GEOID'
                 )
ct_screwed = GeoData(geo_dataframe = gdf_screwed,
                  style={'color': 'black', 'fillColor': '#eb7a34', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                  hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                  name = 'GEOID'
                 )

In [10]:
# dir(ct_therest.popup)

In [11]:
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

# add the layers
m.add_layer(ct_therest)
m.add_layer(ct_screwed)

m.add_control(FullScreenControl())

#ct_screwed.popup = message

## Note: orange census tracts are selected randomly here for testing purposes.

#### Also, map takes a few seconds to load

In [12]:
display(m)

Map(center=[37.248, -119.432], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [13]:
# now let's try zipcodes
gdf_zc = gpd.read_file('data/cb_2019_us_zcta510_500k/')

In [14]:
zipcode_list = pd.read_csv('data/zip-code-list.csv')

In [15]:
gdf_zc['ZCTA5CE10'] = gdf_zc['ZCTA5CE10'].astype(int)

In [16]:
df_zc_ca = pd.merge(zipcode_list, gdf_zc, how='left', left_on='zip_code', right_on='ZCTA5CE10')

In [17]:
len(df_zc_ca)

2664

In [18]:
len(zipcode_list)

2664

In [19]:
df_zc_ca.head(2)

,zip_code,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry
0,90001,90001.0,8600000US90001,90001,9071081.0,0.0,"POLYGON ((-118.26519 33.98923, -118.25644 33.9..."
1,90002,90002.0,8600000US90002,90002,7930602.0,0.0,"POLYGON ((-118.26512 33.96013, -118.25858 33.9..."


In [20]:
gdf_zc_ca = gpd.GeoDataFrame(df_zc_ca, crs="EPSG:4326", geometry='geometry')

In [21]:
# prepping layers for map
ct_zips = GeoData(geo_dataframe = gdf_zc_ca,
                  style={'color': 'black', 'fillColor': 'blue', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                  hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                  name = 'GEOID'
                 )

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

# add the layers
m.add_layer(ct_zips)

m.add_control(FullScreenControl())

display(m)

Map(center=[37.248, -119.432], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…